In [ ]:
# spark-submit project3.py "file:///home/sample.csv" "file:///home/output" 0.5
# spark-submit project3.py "file:///home/testcase1.csv" "file:///home/output" 0.7
# spark-submit project3.py "file:///home/testcase2.csv" "file:///home/output" 0.8
# time spark-submit project3.py "file:///home/testcase1.csv" "file:///home/output" 0.7
# time spark-submit project3.py "file:///home/testcase2.csv" "file:///home/output" 0.8
# testcase 1
# real0m11.479s
# user0m16.043s
# sys0m0.623s
# testcase 2
# real0m28.500s
# user0m27.747s
# sys0m0.688s

背景：相似性连接是数据挖掘中的一项重要任务。 给定一组记录，其目标是找到相似度大于阈值的所有记录对。 记录是一组项目。 例如，在亚马逊中，一条记录可以代表客户在单次交易中购买的一组产品。 该问题可以在不同领域找到各种应用，例如数据清洗和产品推荐。

给定记录集合 R、相似度函数 sim() 和阈值 t，R 上的相似度连接就是从 R 中找到所有记录对 r 和 s，使得 sim(r, s)>=t。 在这个项目中，我们使用 Jaccard 相似度函数来计算相似度

问题定义：在这个项目中，我们仍然将使用客户购买交易日志的电子商务数据集。 数据集中的每条记录都有以下五个字段（请参阅示例数据集）：

InvoiceNo: the unique ID to record one purchase transaction

Description: the name of the item in a transaction (a name can contain multiple characters)

Quantity: the amount of the items purchased

InvoiceDate: the time of the transaction

UnitPrice: the price of a single item

每笔交易都包含一组购买的物品。 例如，在示例数据集中，交易 1 = {A, B, C}，交易 2 = {A, C, DD} 和交易 3 = {A, B, C, DD}。 您的任务是利用 Spark 查找不同年份的所有相似交易对。

输出格式：输出文件包含所有相似交易及其相似之处。 输出格式为“(发票号1,发票号2):相似度值”。 在每对中，交易必须来自不同年份且 InvoiceNo1 < InvoiceNo2。 输出结果不应有重复。 这些对按升序排序（按第一个，然后是第二个）。 给定上述阈值为 0.5 的示例数据集，输出结果应为：
(1,3):0.75
(2,3):0.75
(1,2)：不返回 0.5，因为交易 1 和 2 是同一年.

In [ ]:
import sys
from pyspark import SparkConf, SparkContext
from datetime import datetime

def extract_month_year(date):
    date = date.split(' ')[0]
    split_date = date.split("/")
    new_date = "/".join([split_date[1], split_date[2]])
    return new_date

class project3:
    def run(self, inputpath, outputpath, k):
        conf = SparkConf()
        sc = SparkContext(conf = conf)

        data= sc.textFile(inputpath)
        data = data.map(lambda line: line.split(','))
        
        
        
        
        
        #final_result.coalesce(1).saveAsTextFile(outputPath)
        
if __name__ == '__main__':
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from pyspark.sql.window import Window
from itertools import combinations
from pyspark.sql.types import FloatType

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')


        
        grouped_data = data_filtered.groupBy("InvoiceNo", "Year").agg(concat_ws(" ", collect_list("Description")).alias("Description"))
        data_with_value = grouped_data.withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))

        # Explode the Description column to create a new row for each element
        exploded_data = data_with_value.withColumn("Key", explode(split("Description", " ")))

        data_partition_by_prefix = exploded_data.select("Key", "InvoiceNo_Items", "Year")

        def jaccard_similarity(set1, set2):
            intersection_size = len(set(set1).intersection(set2))
            union_size = len(set(set1).union(set2))
            return intersection_size / union_size if union_size != 0 else 0

        self_joined_data = data_partition_by_prefix.alias("df1").join(data_partition_by_prefix.alias("df2"), "Key")
        
        
        renamed_data = self_joined_data.select(col("df1.Key").alias("Key"),
                     col("df1.InvoiceNo_Items").alias("InvoiceNo_Items_1"),
                     col("df1.Year").alias("Year_1"),
                     col("df2.InvoiceNo_Items").alias("InvoiceNo_Items_2"),
                     col("df2.Year").alias("Year_2"),
                     regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                     regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
                )
        
        splitted_data = renamed_data.select("Key", 
                     substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                     "Year_1",
                     substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                     "Year_2",
                     "InvoiceNo_1",
                     "InvoiceNo_2"
                )

        paired_data = splitted_data.withColumn("pair", concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")")))
        
        filtered_data = paired_data.filter(col("Year_1") != col("Year_2"))
        
        filtered_data.show()
        spark.stop()

if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])




# Efficiency/Similarity Two-step Solution

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = ( 
             data
            .select("InvoiceNo", "Description","InvoiceDate")
            .withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
            .drop('InvoiceDate')
        )
        
        grouped_data = (
             data.groupBy("InvoiceNo", "Year")
             .agg(concat_ws(" ", collect_list("Description")).alias("Description"))
        )
            
        data_partition_by_prefix = (
             grouped_data
             .withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))
             .withColumn("Key", explode(split("Description", " ")))
             .select("Key", "InvoiceNo_Items", "Year")
        )


        def jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return (intersection) / union

        self_joined_data = (
            data_partition_by_prefix
            .alias("df1")
            .join(data_partition_by_prefix
            .alias("df2"), "Key")
        )
        
        
        renamed_data = (
            self_joined_data
            .select(col("df1.Key").alias("Key"),
                     col("df1.InvoiceNo_Items").alias("InvoiceNo_Items_1"),
                     col("df1.Year").alias("Year_1"),
                     col("df2.InvoiceNo_Items").alias("InvoiceNo_Items_2"),
                     col("df2.Year").alias("Year_2"),
                     regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                     regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
            )
        )
        
        splitted_data = (
            renamed_data
            .select("Key", 
                     substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                     "Year_1",
                     substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                     "Year_2",
                     "InvoiceNo_1",
                     "InvoiceNo_2"
            )
        )
        
        jaccard_similarity_udf = udf(jaccard_similarity, FloatType())
        sort_tuple_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())

        filtered_data = (
            splitted_data
            .withColumn("pair", 
                        concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
            )
            .filter(col("Year_1") != col("Year_2")).drop("InvoiceNo_1","InvoiceNo_2")
            .withColumn("sorted_pair", sort_tuple_udf("pair"))
            .drop("pair")
        )
        
        result_df = (
            filtered_data
            .withColumn("Similarity", 
                        jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
            )
        )
        
        result_df = (
            result_df
            .select('sorted_pair','Similarity')
            .dropDuplicates(["sorted_pair", "Similarity"])
            .orderBy('sorted_pair')
            .filter(col("Similarity") >= k)
        )
        
        result_format_df = (
            result_df
            .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
            .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        )
        result_format_df = result_format_df.withColumn("result", regexp_replace("result", ",\\s*", ","))
        #sorted_result_df.show()
        
        result_format_df.write.format("csv").option("header", "false").save(outputpath)
        
        spark.stop()

if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])




# Efficiency/Similarity Exact Solution

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from pyspark.sql.window import Window

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')
        
        grouped_data = data_filtered.groupBy("InvoiceNo", "Year").agg(concat_ws(" ", collect_list("Description")).alias("Description"))
        data_with_value = grouped_data.withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))

        cal_token_frequencies = data_with_value.select('InvoiceNo','Description').withColumn("tokens", split("Description", " "))
        df_exploded = cal_token_frequencies.select("InvoiceNo", explode("tokens").alias("token"))
        token_frequencies = df_exploded.groupBy("token").agg(count("token").alias("frequency")).orderBy('frequency')
        tokens = token_frequencies.select("token")
       
        data_original = data_with_value.select('InvoiceNo','Description','InvoiceNo_Items','Year').orderBy('InvoiceNo')
        # Calculate prefixes for each record
        #windowSpec = Window.partitionBy("tokens")
        #df_swapped = df_swapped.withColumn("prefix", concat_ws(",", collect_list("token").over(window_spec)))
        
        #token_frequencies.show()
        #result_df = data_original.groupBy(tokens).agg(collect_list("InvoiceNo"), collect_list("Description"))
        #sorted_result_df.write.format("csv").option("header", "false").save(outputpath)
        token_list = [row.token for row in tokens.collect()]
        # def find_prefix(description):
        #     prefixes = []
        #     for token in token_list:
        #         if token in description:
        #             prefixes.append(token)
        #     return prefixes
        
        def find_prefix(description):
            prefixes = []
            for token in token_list:
                if token in description:
                    prefixes.append(token)
            prefixes.pop()
            return prefixes

        find_prefix_udf = udf(find_prefix, ArrayType(StringType()))
        #data_with_prefix = data_original.withColumn("Prefixes", find_prefix_udf(col("Description")))
        
        data_with_remaining_tokens = data_original.withColumn("Tokens", find_prefix_udf(col("Description")))
        
        exploded_data = data_with_remaining_tokens.select(
            explode("Tokens").alias("Key"),
            col("InvoiceNo_Items"),
            col("Year")
        )

        self_joined_data = (
            exploded_data
            .alias("df1")
            .join(exploded_data
            .alias("df2"), "Key")
        )
        renamed_data = (
            self_joined_data
            .select(col("df1.Key").alias("Key"),
                     col("df1.InvoiceNo_Items").alias("InvoiceNo_Items_1"),
                     col("df1.Year").alias("Year_1"),
                     col("df2.InvoiceNo_Items").alias("InvoiceNo_Items_2"),
                     col("df2.Year").alias("Year_2"),
                     regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                     regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
            )
        )
        
        splitted_data = (
            renamed_data
            .select("Key", 
                     substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                     "Year_1",
                     substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                     "Year_2",
                     "InvoiceNo_1",
                     "InvoiceNo_2"
            )
        )
        data = splitted_data.filter(col("Year_1") != col("Year_2"))
        
        def jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return (intersection) / union

        jaccard_similarity_udf = udf(jaccard_similarity, FloatType())
        sort_tuple_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())
        result_df = (
            data
            .withColumn("Similarity", 
                        jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
            )
        )

        result_df = (
            result_df
            .withColumn("pair", 
                        concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
            )
            .drop("InvoiceNo_1","InvoiceNo_2")
            .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
            .withColumn("sorted_pair", sort_tuple_udf("pair"))
            .drop("pair")
        )
        
        result_df = (
            result_df
            .select('sorted_pair','Similarity')
            .dropDuplicates(["sorted_pair", "Similarity"])
            .orderBy('sorted_pair')
            .filter(col("Similarity") >= k)
        )

        result_format_df = (
            result_df
            .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
            .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        )

        exploded_data.show()
        #result_format_df.write.format("csv").option("header", "false").save(outputpath)
        spark.stop()

if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])




# Efficiency/Similarity Exact Solution - But join or crossjoin

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from math import floor

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')
        
        # Concatenate the items group by InvoiceNo and year then put them into one column
        grouped_data = (
                   data_filtered
                   .groupBy("InvoiceNo", "Year")
                   .agg(concat_ws(" ", collect_list("Description")).alias("Description"))
                )
        
        data_combined_InvoiceNo_Items = (
                   grouped_data
                   .withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))   
                )
        
        # Calculate the tokens' frequency and get the sorted token list according to their frequencies
        cal_token_frequencies = (
                    data_combined_InvoiceNo_Items
                    .select('InvoiceNo','Description')
                    .withColumn("tokens", split("Description", " "))
                )
        df_exploded = (
                    cal_token_frequencies
                    .select("InvoiceNo", explode("tokens").alias("token"))
                )
        token_frequencies = (
                    df_exploded
                    .groupBy("token")
                    .agg(count("token").alias("frequency"))
                    .orderBy('frequency')
                )
        tokens = token_frequencies.select("token")
        token_list = [row.token for row in tokens.collect()]

        # Define a function to calculate the length of prefixes
        def calculate_prefix(description, k):
            description = set(description.split())
            length = floor(len(description) - len(description)*float(k) + 1)
            return length
        
        # udf transform
        calculate_prefix_length_udf = udf(calculate_prefix)

        df_prefix_length = (
                    data_combined_InvoiceNo_Items
                    .withColumn("Prefix_Length", calculate_prefix_length_udf(col("Description"), lit(k)))
                )
        
        # Define a function to find corresponding prefixes according to their legnths
        def find_prefix(description,prefix_length):
            prefixes = []
            count = 0
            for token in token_list:
                if count < int(prefix_length):
                    if token in description:
                        prefixes.append(token)
                        count += 1
            return prefixes
        
        # udf transform
        find_prefix_udf = udf(find_prefix, ArrayType(StringType()))  
        
        # Obtain the prefixes using the function above
        data_get_prefixes = (
                    df_prefix_length
                    .withColumn("Prefixes", find_prefix_udf(col("Description"),col('Prefix_Length')))
                )
        
        # Explode the prefixes to get new keys for records
        exploded_prefixes = (
                    data_get_prefixes
                    .select(explode("Prefixes").alias("Key"),col("InvoiceNo_Items"),col("Year"))
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
                )
        
        # Self-join the data for calculating the similarity
        self_joined_data = (
                    exploded_prefixes
                    .alias("df1")
                    .join(exploded_prefixes
                    .alias("df2"), "Key")
                )
        
        # Rearrange the data 
        rearrange_data = (
                    self_joined_data
                    .select(col("df1.Key").alias("Key"),
                             col("df1.InvoiceNo_Items").alias("InvoiceNo_Items_1"),
                             col("df1.Year").alias("Year_1"),
                             col("df2.InvoiceNo_Items").alias("InvoiceNo_Items_2"),
                             col("df2.Year").alias("Year_2"),
                             regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                             regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
                )
        )
        
        # Remove the same year records
        filter_year_data = (
                    rearrange_data
                    .select("Key", 
                             substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                             "Year_1",
                             substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                             "Year_2",
                             "InvoiceNo_1",
                             "InvoiceNo_2"
                )
                    
        )
        filter_year_data = filter_year_data.filter(col("Year_1") != col("Year_2"))
        
        # Define a function to calculate the jaccard similarity between two item lists
        def calculate_jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return (intersection) / union

        # udf transform
        jaccard_similarity_udf = udf(calculate_jaccard_similarity, FloatType())
        
        # # udf transform
        sort_pair_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())
        
        # Calculate the similarity between two item columns
        similarity_data = (
                    filter_year_data
                    .withColumn("Similarity", 
                    jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
                )
        )
        
        # Combine two InvoiceNo into a pair and drop redundant columns
        similarity_pair_data = (
                    similarity_data
                    .withColumn("pair", 
                    concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
                )
                    .drop("InvoiceNo_1","InvoiceNo_2")
                    .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
                    .withColumn("sorted_pair", sort_pair_udf("pair"))
                    .drop("pair")
        )
        
        # Drop duplicated records and filter records by threshold
        similarity_pair_filtered_data = (
                    similarity_pair_data
                    .select('sorted_pair','Similarity')
                    .dropDuplicates(["sorted_pair", "Similarity"])
                    .orderBy('sorted_pair')
                    .filter(col("Similarity") >= k)
        )
        
        # Obtain the final result by concatenating pair and similarity with the symbol colon
        final_result = (
                    similarity_pair_filtered_data
                    .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
                    .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        )

        # Remove the empty space in the pairs
        final_result = final_result.withColumn("result", regexp_replace("result", ",\\s*", ","))
        
        # Write the final result into csv file
        final_result.write.format("csv").option("header", "false").save(outputpath)
        
if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])
      

In [ ]:
def find_prefix(description,prefix_length):
    prefixes = []
    for token in token_list:
        for i in range(prefix_length):
            if token in description:
                prefixes.append(token)
    return prefixes

In [2]:
a = ['XMD','G','G','G','PV','VAT','SW','MOU','G']
b = ['G','PV','XMD','G','MOU','SW','G']

In [4]:
a = set(a)
b = set(b)

In [12]:
a

{'G', 'MOU', 'PV', 'SW', 'VAT', 'XMD'}

In [15]:
b

{'G', 'MOU', 'PV', 'SW', 'XMD'}

In [9]:
len(a.intersection(b))

5

In [10]:
len(a.union(b))

6

In [11]:
len(a.intersection(b)) / len(a.union(b))

0.8333333333333334

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from math import floor

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')
        
        # Concatenate the items group by InvoiceNo and year then put them into one column
        grouped_data = (
                   data_filtered
                   .groupBy("InvoiceNo", "Year")
                   .agg(concat_ws(" ", collect_list("Description")).alias("Description"))
                )
        
        data_combined_InvoiceNo_Items = (
                   grouped_data
                   .withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))   
                )
        
        # Calculate the tokens' frequency and get the sorted token list according to their frequencies
        cal_token_frequencies = (
                    data_combined_InvoiceNo_Items
                    .select('InvoiceNo','Description')
                    .withColumn("tokens", split("Description", " "))
                )
        df_exploded = (
                    cal_token_frequencies
                    .select("InvoiceNo", explode("tokens").alias("token"))
                )
        token_frequencies = (
                    df_exploded
                    .groupBy("token")
                    .agg(count("token").alias("frequency"))
                    .orderBy('frequency')
                )
        tokens = token_frequencies.select("token")
        token_list = [row.token for row in tokens.collect()]

        # Define a function to calculate the length of prefixes
        def calculate_prefix(description, k):
            description = set(description.split())
            length = floor(len(description) - len(description)*float(k) + 1)
            return length
        
        # udf transform
        calculate_prefix_length_udf = udf(calculate_prefix)

        df_prefix_length = (
                    data_combined_InvoiceNo_Items
                    .withColumn("Prefix_Length", calculate_prefix_length_udf(col("Description"), lit(k)))
                )
        
        # Define a function to find corresponding prefixes according to their legnths
        def find_prefix(description,prefix_length):
            prefixes = []
            count = 0
            for token in token_list:
                if count < int(prefix_length):
                    if token in description:
                        prefixes.append(token)
                        count += 1
            return prefixes
        
        # udf transform
        find_prefix_udf = udf(find_prefix, ArrayType(StringType()))  
        
        # Obtain the prefixes using the function above
        data_get_prefixes = (
                    df_prefix_length
                    .withColumn("Prefixes", find_prefix_udf(col("Description"),col('Prefix_Length')))
                )
        
        # Explode the prefixes to get new keys for records
        df1 = (
                    data_get_prefixes
                    .select(explode("Prefixes").alias("Key"),col("InvoiceNo_Items_1"),col("Year_1"))
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        
        df2 = (
                    data_get_prefixes
                    .select(explode("Prefixes").alias("Key"),col("InvoiceNo_Items_2"),col("Year_2"))
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        # Define a function to calculate the jaccard similarity between two item lists
        def calculate_jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return float((intersection) / union)

        # udf transform
        jaccard_similarity_udf = udf(calculate_jaccard_similarity, FloatType())
        
        
        
        # Self-join the data for calculating the similarity
        self_joined_data = (
                    df1
                    .crossJoin(df2.select('InvoiceNo_Items_2','Year_2'))
                    .select(
                           col("Key"),
                           col('InvoiceNo_Items_1'),
                           col('InvoiceNo_Items_2'),
                           col('Year_1'),
                           col('Year_2'),
                           regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                           regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
                    )
                    .select("Key", 
                             substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                             "Year_1",
                             substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                             "Year_2",
                             "InvoiceNo_1",
                             "InvoiceNo_2"
                    )
                    .filter(col("Year_1") != col("Year_2"))
                    .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
                    .withColumn(
                            "Similarity", 
                            jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
                        )
                    .filter(col('Similarity') >= k)  
                    .select("InvoiceNo_1", "InvoiceNo_2", "Similarity")
                    
        )
        self_joined_data.show()
        # # Rearrange the data 
        # rearrange_data = (
        #             self_joined_data
        #             .select(col("df1.Key").alias("Key"),
        #                      col("df1.InvoiceNo_Items").alias("InvoiceNo_Items_1"),
        #                      col("df1.Year").alias("Year_1"),
        #                      col("df2.InvoiceNo_Items").alias("InvoiceNo_Items_2"),
        #                      col("df2.Year").alias("Year_2"),
        #                      regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
        #                      regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
        #         )
        # )
        
        # # Remove the same year records
        # filter_year_data = (
        #             rearrange_data
        #             .select("Key", 
        #                      substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
        #                      "Year_1",
        #                      substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
        #                      "Year_2",
        #                      "InvoiceNo_1",
        #                      "InvoiceNo_2"
        #         )
                    
        # )
        # filter_year_data = filter_year_data.filter(col("Year_1") != col("Year_2"))
        
        # # # udf transform
        # sort_pair_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())
        
        # # Calculate the similarity between two item columns
        # similarity_data = (
        #             filter_year_data
        #             .withColumn("Similarity", 
        #             jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
        #         )
        # )
        
        # # Combine two InvoiceNo into a pair and drop redundant columns
        # similarity_pair_data = (
        #             similarity_data
        #             .withColumn("pair", 
        #             concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
        #         )
        #             .drop("InvoiceNo_1","InvoiceNo_2")
        #             .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
        #             .withColumn("sorted_pair", sort_pair_udf("pair"))
        #             .drop("pair")
        # )
        
        # # Drop duplicated records and filter records by threshold
        # similarity_pair_filtered_data = (
        #             similarity_pair_data
        #             .select('sorted_pair','Similarity')
        #             .dropDuplicates(["sorted_pair", "Similarity"])
        #             .orderBy('sorted_pair')
        #             .filter(col("Similarity") >= k)
        # )

        # # Obtain the final result by concatenating pair and similarity with the symbol colon
        # final_result = (
        #             similarity_pair_filtered_data
        #             .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
        #             .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        # )

        # Write the final result into csv file
        #final_result.write.format("csv").option("header", "false").save(outputpath)
        
if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])
      

# Efficiency/Similarity Exact Solution - But join or crossjoin

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from math import floor

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')
        
        # Concatenate the items group by InvoiceNo and year then put them into one column
        grouped_data = (
                   data_filtered
                   .groupBy("InvoiceNo", "Year")
                   .agg(concat_ws(" ", collect_list("Description")).alias("Description"))
                )
        
        data_combined_InvoiceNo_Items = (
                   grouped_data
                   .withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))   
                )
        
        # Calculate the tokens' frequency and get the sorted token list according to their frequencies
        cal_token_frequencies = (
                    data_combined_InvoiceNo_Items
                    .select('InvoiceNo','Description')
                    .withColumn("tokens", split("Description", " "))
                )
        df_exploded = (
                    cal_token_frequencies
                    .select("InvoiceNo", explode("tokens").alias("token"))
                )
        token_frequencies = (
                    df_exploded
                    .groupBy("token")
                    .agg(count("token").alias("frequency"))
                    .orderBy('frequency')
                )
        tokens = token_frequencies.select("token")
        token_list = [row.token for row in tokens.collect()]

        # Define a function to calculate the length of prefixes
        def calculate_prefix(description, k):
            description = set(description.split())
            length = floor(len(description) - len(description)*float(k) + 1)
            return length
        
        # udf transform
        calculate_prefix_length_udf = udf(calculate_prefix)

        df_prefix_length = (
                    data_combined_InvoiceNo_Items
                    .withColumn("Prefix_Length", calculate_prefix_length_udf(col("Description"), lit(k)))
                )
        
        # Define a function to find corresponding prefixes according to their legnths
        def find_prefix(description,prefix_length):
            prefixes = []
            count = 0
            for token in token_list:
                if count < int(prefix_length):
                    if token in description:
                        prefixes.append(token)
                        count += 1
            return prefixes
        
        # udf transform
        find_prefix_udf = udf(find_prefix, ArrayType(StringType()))  
        
        # Obtain the prefixes using the function above
        data_get_prefixes = (
                    df_prefix_length
                    .withColumn("Prefixes", find_prefix_udf(col("Description"),col('Prefix_Length')))
                )
        
        # Explode the prefixes to get new keys for records
        df1 = (
                    data_get_prefixes
                    .select(explode("Prefixes").alias("Key"),col("InvoiceNo_Items").alias('InvoiceNo_Items_1'),col("Year").alias('Year_1'))
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        
        df2 = (
                    data_get_prefixes
                    .select(explode("Prefixes").alias("Key"),col("InvoiceNo_Items").alias('InvoiceNo_Items_2'),col("Year").alias('Year_2'))
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        # Define a function to calculate the jaccard similarity between two item lists
        def calculate_jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return float((intersection) / union)

        # udf transform
        jaccard_similarity_udf = udf(calculate_jaccard_similarity, FloatType())
        
        
        # Self-join the data for calculating the similarity
        self_joined_data = (
                    df1
                    .crossJoin(df2.select('InvoiceNo_Items_2','Year_2'))
                    .select(
                           col("Key"),
                           col('InvoiceNo_Items_1'),
                           col('InvoiceNo_Items_2'),
                           col('Year_1'),
                           col('Year_2'),
                           regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                           regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
                    )
                    .select("Key", 
                             substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                             "Year_1",
                             substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                             "Year_2",
                             "InvoiceNo_1",
                             "InvoiceNo_2"
                    )
                    .filter(col("Year_1") != col("Year_2"))
                    .drop('Key','Year_1','Year_2')
                    .withColumn(
                            "Similarity", 
                            jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
                        )
                    .drop('InvoiceNo_Items_1','InvoiceNo_Items_2')
                    .filter(col('Similarity') >= k)  
                    .select("InvoiceNo_1", "InvoiceNo_2", "Similarity")    
        )
               
        # udf transform
        sort_pair_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())
        
        
        # Combine two InvoiceNo into a pair and drop redundant columns
        similarity_pair_data = (
                    self_joined_data
                    .withColumn("pair", 
                    concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
                )
                    .drop("InvoiceNo_1","InvoiceNo_2")
                    .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
                    .withColumn("sorted_pair", sort_pair_udf("pair"))
                    .drop("pair")
        )
        
        # Drop duplicated records and filter records by threshold
        similarity_pair_filtered_data = (
                    similarity_pair_data
                    .select('sorted_pair','Similarity')
                    .dropDuplicates(["sorted_pair", "Similarity"])
                    .orderBy('sorted_pair')
                    
        )
        
        # Obtain the final result by concatenating pair and similarity with the symbol colon
        final_result = (
                    similarity_pair_filtered_data
                    .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
                    .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        )

        final_result = final_result.withColumn("result", regexp_replace("result", ",\\s*", ","))
        # Write the final result into csv file
        #final_result.show()
        #final_result.write.format("csv").option("header", "false").save(outputpath)
        
if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])
      

In [ ]:
+-----+
|token|
+-----+
|   GU|
|  LEW|
|   YA|
|   EH|
|    A|
|    T|
|  EUI|
|   PV|
|   YI|
|  MNJ|
|   HD|
|   HO|
|  MJB|
|  MOU|
|  XMD|
|    D|
|   NX|
|   ZH|
|   EK|
|    I|
+-----+

# Final Result - Submitted

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys
from math import floor

class project3:           
    def run(self, inputpath, outputpath, k):
        spark = SparkSession.builder.master("local").appName("project3_df").getOrCreate()
        
        # Read the input CSV file and give the column names for them
        data = spark.read.csv(inputpath).toDF("InvoiceNo", "Description", "Quantity", "InvoiceDate", "UnitPrice")
        
        # Select needed columns
        data = data.select("InvoiceNo", "Description","InvoiceDate")
        
        # Extract month and year from the InvoiceDate field 
        data_filtered = data.withColumn("Year",  split(split(data["InvoiceDate"], ' ')[0], '/')[2])
        data_filtered = data_filtered.drop('InvoiceDate')
        
        # Concatenate the items group by InvoiceNo and year then put them into one column
        grouped_data = (
                   data_filtered
                   .groupBy("InvoiceNo", "Year")
                   .agg(concat_ws(" ", collect_list("Description")).alias("Description"))
                )
        
        data_combined_InvoiceNo_Items = (
                   grouped_data
                   .withColumn("InvoiceNo_Items", concat_ws(",", col("InvoiceNo"), col("Description")))   
                )
        
        # Calculate the tokens' frequency and get the sorted token list according to their frequencies
        cal_token_frequencies = (
                    data_combined_InvoiceNo_Items
                    .select('InvoiceNo','Description')
                    .withColumn("tokens", split("Description", " "))
                )
        df_exploded = (
                    cal_token_frequencies
                    .select("InvoiceNo", explode("tokens").alias("token"))
                )
        token_frequencies = (
                    df_exploded
                    .groupBy("token")
                    .agg(count("token").alias("frequency"))
                    .orderBy('frequency')
                )
        tokens = token_frequencies.select("token")
        token_list = [row.token for row in tokens.collect()]

        # Define a function to calculate the length of prefixes
        def calculate_prefix(description, k):
            description = set(description.split())
            length = floor(len(description) - len(description)*float(k) + 1)
            return length
        
        # udf transform
        calculate_prefix_length_udf = udf(calculate_prefix)
        
        # Calculate the length of prefixes for each record
        df_prefix_length = (
                    data_combined_InvoiceNo_Items
                    .withColumn("Prefix_Length", calculate_prefix_length_udf(col("Description"), lit(k)))
                )
        
        # Define a function to find corresponding prefixes according to their legnths
        def find_prefix(description,prefix_length):
            prefixes = []
            count = 0
            for token in token_list:
                if count < int(prefix_length):
                    if token in description.split():
                        prefixes.append(token)
                        count += 1
            return prefixes
        
        # udf transform
        find_prefix_udf = udf(find_prefix, ArrayType(StringType()))  
        
        # Obtain the prefixes using the function above
        data_get_prefixes = (
                    df_prefix_length
                    .withColumn("Prefixes", find_prefix_udf(col("Description"),col('Prefix_Length')))
                )
        
        # Create two dataframes for self-join, each one explodes the prefixes to get new keys for records
        df1 = (
                    data_get_prefixes
                    .select(
                        explode("Prefixes").alias("Key"),
                        col("InvoiceNo_Items").alias('InvoiceNo_Items_1'),col("Year").alias('Year_1')
                    )
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        
        df2 = (
                    data_get_prefixes
                    .select(
                        explode("Prefixes").alias("Key"),
                        col("InvoiceNo_Items").alias('InvoiceNo_Items_2'),col("Year").alias('Year_2')
                    )
                    .drop('InvoiceNo')
                    .drop('Description')
                    .drop('Prefix_Length')
        )
        # Define a function to calculate the jaccard similarity between two item lists
        def calculate_jaccard_similarity(s1, s2):
            set1 = set(s1.split())
            set2 = set(s2.split())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return float((intersection) / union)

        # udf transform
        jaccard_similarity_udf = udf(calculate_jaccard_similarity, FloatType())
        
        # Self-join the data for calculating the similarity and reformat the expected dataframe output 
        self_joined_data = (
                    df1
                    .join(df2,'Key')
                    .select(
                           col("Key"),
                           col('InvoiceNo_Items_1'),
                           col('InvoiceNo_Items_2'),
                           col('Year_1'),
                           col('Year_2'),
                           regexp_extract("InvoiceNo_Items_1", "\\d+", 0).alias("InvoiceNo_1"),
                           regexp_extract("InvoiceNo_Items_2", "\\d+", 0).alias("InvoiceNo_2")
                    )
                    .select("Key", 
                             substring_index("InvoiceNo_Items_1", ",", -1).alias("InvoiceNo_Items_1"),
                             "Year_1",
                             substring_index("InvoiceNo_Items_2", ",", -1).alias("InvoiceNo_Items_2"),
                             "Year_2",
                             "InvoiceNo_1",
                             "InvoiceNo_2"
                    )
                    .filter(col("Year_1") != col("Year_2"))
                    .drop('Key','Year_1','Year_2')
                    .withColumn(
                            "Similarity", 
                            jaccard_similarity_udf(col("InvoiceNo_Items_1"), col("InvoiceNo_Items_2"))
                    )
                    .drop('InvoiceNo_Items_1','InvoiceNo_Items_2')
                    .filter(col('Similarity') >= k)  
                    .select("InvoiceNo_1", "InvoiceNo_2", "Similarity")    
        )
               
        # udf transform
        sort_pair_udf = udf(lambda pair: str(tuple(sorted(eval(pair)))), StringType())
        
        # Combine two InvoiceNo into a pair and drop redundant columns
        similarity_pair_data = (
                    self_joined_data
                    .withColumn(
                            "pair", 
                            concat(lit("("), col("InvoiceNo_1"), lit(","), col("InvoiceNo_2"), lit(")"))
                    )
                    .drop("InvoiceNo_1","InvoiceNo_2")
                    .drop('Key','InvoiceNo_Items_1','Year_1','InvoiceNo_Items_2','Year_2')
                    .withColumn("sorted_pair", sort_pair_udf("pair"))
                    .drop("pair")
        )
        
        # Drop duplicated records and filter records by threshold
        similarity_pair_filtered_data = (
                    similarity_pair_data
                    .select('sorted_pair','Similarity')
                    .dropDuplicates(["sorted_pair", "Similarity"])
                    .orderBy('sorted_pair')  
        )
        
        # Obtain the final result by concatenating pair and similarity with the symbol colon
        final_result = (
                    similarity_pair_filtered_data
                    .select(concat_ws(":", col("sorted_pair"), col("Similarity")).alias('result'))
                    .orderBy(regexp_extract("result", r"\((\d+),", 1).cast("int"))
        )
        
        # Remove the empty space in the pairs
        final_result = final_result.withColumn("result", regexp_replace("result", ",\\s*", ","))
        
        # Write the final result into csv file
        final_result.write.format("csv").option("header", "false").save(outputpath)
        
        # testcase 1: rt: 0m10.627s   testcase 2: rt: 0m22.432s
if __name__ == "__main__":
    project3().run(sys.argv[1], sys.argv[2], sys.argv[3])
      